In [2]:
#large cap stocks
"""
pre-market stock scanner
Technical Requirements:
1-pre-market top gainers
change from close >= 15%
price >= $1.00
Volume >= 20,000 (shares)
Market Cap = $N/A (share price * number of shares outstanding)


2-Mid-big cap gainers
change from the close >= 2%
price >= $5.00 <= $400.00 (could do $500.00)
Volume >= 20000 (shares)
Float <= (float=total outstanding shares - restricted shares)
Market Cap = $5000000000 (5 billion)
"""

'\npre-market stock scanner\nTechnical Requirements:\n1-pre-market top gainers\nchange from close >= 15%\nprice >= $1.00\nVolume >= 20,000 (shares)\nMarket Cap = $N/A (share price * number of shares outstanding)\n\n\n2-Mid-big cap gainers\nchange from the close >= 2%\nprice >= $5.00 <= $400.00 (could do $500.00)\nVolume >= 20000 (shares)\nFloat <= (float=total outstanding shares - restricted shares)\nMarket Cap = $5000000000 (5 billion)\n'

In [3]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv #pip3 install python-dotenv

# load secrets
load_dotenv()

True

In [4]:
#variables
screener_col_names = ['ticker',
                      'stock_price',
                      'float_shares',
                      'short_float', #more significant for small cap stocks
                      'change_close',
                      'volume_today',
                      'volume_avg_5d',
                      'market_cap',
                      'insitutional_ownership',
                      'sector',
                      'company_name']

In [5]:
def get_stock_data(symbol):
    iex_key = os.getenv("IEX_API_KEY") 
    
    #define and append endpoints below
    quote_url = f'https://cloud.iexapis.com/v1/stock/{symbol}/quote?token={iex_key}'
    company_url = f'https://cloud.iexapis.com/v1/stock/{symbol}/company?token={iex_key}'
    float_url = f'https://cloud.iexapis.com/v1/data-points/{symbol}/FLOAT?token={iex_key}'
    
    #execute api calls
    quote_data = requests.get(quote_url).json()
    company_data = requests.get(company_url).json()
    float_data = requests.get(float_url).json()

    #data mapping
    stock_data = {
        'ticker': symbol,
        'stock_price': quote_data['latestPrice'],
        'change_close': quote_data['change'], #KEY-Price Gap
        'change_close_pct': quote_data['changePercent'],
        'volume_today': quote_data['latestVolume'],
        'volume_avg_5d': None, 
        'market_cap': quote_data['marketCap'],
        'float': float_data,  
        'float_pct': None, 
        'volatility': None,
        'liquidity': None,
        'news': None,
        'sector': company_data['sector'],
        'industry': company_data['industry'],
        'company_name': company_data['companyName'],
        'latestUpdate': quote_data['latestUpdate']
    }

    return stock_data

#execute testing
ticker = 'PLTR'
stock_data = get_stock_data(ticker)
print(stock_data)

{'ticker': 'PLTR', 'stock_price': 17.99, 'change_close': -0.21, 'change_close_pct': -0.01154, 'volume_today': 133808372, 'volume_avg_5d': None, 'market_cap': 36227524446, 'float': 920329412, 'float_pct': None, 'volatility': None, 'liquidity': None, 'news': None, 'sector': 'Information', 'industry': 'Software Publishers', 'company_name': 'Palantir Technologies Inc', 'latestUpdate': 1691438402092}


In [6]:
#create df from data
stock_df = pd.DataFrame([stock_data])

#print df
stock_df

,ticker,stock_price,change_close,change_close_pct,volume_today,volume_avg_5d,market_cap,float,float_pct,volatility,liquidity,news,sector,industry,company_name,latestUpdate
0,PLTR,17.99,-0.21,-0.01154,133808372,None,36227524446,920329412,None,None,None,None,Information,Software Publishers,Palantir Technologies Inc,1691438402092
